In [4]:
from NovelSwarmBehavior.novel_swarms.behavior.AngularMomentum import AngularMomentumBehavior
from NovelSwarmBehavior.novel_swarms.behavior.AverageSpeed import AverageSpeedBehavior
from NovelSwarmBehavior.novel_swarms.behavior.GroupRotationBehavior import GroupRotationBehavior
from NovelSwarmBehavior.novel_swarms.behavior.RadialVariance import RadialVarianceBehavior
from NovelSwarmBehavior.novel_swarms.behavior.ScatterBehavior import ScatterBehavior
from NovelSwarmBehavior.novel_swarms.behavior.SensorOffset import GeneElementDifference
from NovelSwarmBehavior.novel_swarms.config.AgentConfig import DiffDriveAgentConfig
from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.sensors.GenomeDependentSensor import GenomeBinarySensor
from NovelSwarmBehavior.novel_swarms.sensors.SensorSet import SensorSet
from generation.evolution import ModifiedHaltingEvolution, ModifiedNoveltyArchieve
from generation.halted_evolution import HaltedEvolution
from data.swarmset import SwarmDataset, DataBuilder
import numpy as np
import pygame

def evolution_controller():
    sensors = SensorSet([
        GenomeBinarySensor(genome_id=8, draw=False),
        GenomeBinarySensor(genome_id=9, draw=False)
    ])

    agent_config = DiffDriveAgentConfig(
        sensors=sensors,
        seed=None,
    )

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=((1/3) * np.pi), _min=-((2/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
        GeneRule(_max=((2/3) * np.pi), _min=-((1/3) * np.pi), mutation_step=(np.pi/2), round_digits=4),
    ]

    phenotype = [
        AverageSpeedBehavior(),
        AngularMomentumBehavior(),
        RadialVarianceBehavior(),
        ScatterBehavior(),
        GroupRotationBehavior(),
        # GeneElementDifference(8, 9)
    ]

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=24,
        seed=None,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=50,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=2500,
        display_novelty=True,
        save_archive=True,
        show_gui=True,
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = ModifiedHaltingEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution, screen

def build_random_dataset():
    halted_evolution, screen = evolution_controller()
    baseline_data = DataBuilder("data/full-dual-sensors", steps=1200, agents=30, ev=halted_evolution, screen=screen)
    baseline_data.create()
    baseline_data.evolution.close()

# import time
# t_1 = time.time()
# build_random_dataset()
# print(f"TIME TO DATASET: {time.time() - t_1}")

# Embed into Latent Space

In [5]:

from generation.evolution import ModifiedHaltingEvolution
from scipy import ndimage
import random

def batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images):
    optim.zero_grad()
    anchor_out, pos_out, neg_out = network.batch_network_from_numpy(anchor_images, pos_images, neg_images)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def update_network(network, loss_fn, optim, anchor_image, pos_image, neg_image, with_transforms=False):
    optim.zero_grad()
    if with_transforms:
        anchor_out, pos_out, neg_out = network.network_with_transforms(anchor_image, pos_image, neg_image)
    else:
        anchor_out, pos_out, neg_out = network.network_from_numpy(anchor_image, pos_image, neg_image)
    loss = loss_fn(anchor_out, pos_out, neg_out)
    loss.backward()
    optim.step()
    return loss

def pretraining(data, network, loss_fn, optim, data_cutoff=None, data_size=500):
    if data_cutoff is None:
        data_cutoff = len(data) - 1
    samples = np.random.random_integers(0, data_cutoff, (data_size, 2))
    total_loss = 0
    total_updates = 0
    for i, s in enumerate(samples):
        if i % (data_size // 20) == 0:
            print(f"Unsupervised Training.. {(i * 100) / data_size}")
        anchor_image = data[s[0]][0]
        pos_image = data[s[0]][0]

        pos_images = np.stack([
            [ndimage.rotate(pos_image, 90)],
            [ndimage.rotate(pos_image, 180)],
            [ndimage.rotate(pos_image, 270)],
            # ndimage.gaussian_filter(pos_image, 1),
            # ndimage.gaussian_filter(pos_image, 3),
            # ndimage.gaussian_filter(pos_image, 5),
        ])

        anchor_images = np.stack([[anchor_image] for _ in pos_images])
        neg_images = np.stack([ [data[random.randint(0, data_cutoff)][0]] for _ in pos_images])

        loss = batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images)
        total_loss += loss.item()
        total_updates += 1

    return total_loss / total_updates


def train(network, loss_fn, optim, anchor_img, pos_img, neg_img):

    pos_images = np.stack([
        [ndimage.rotate(pos_img, 90)],
        [ndimage.rotate(pos_img, 180)],
        [ndimage.rotate(pos_img, 270)],
        # ndimage.gaussian_filter(pos_img, 1),
        # ndimage.gaussian_filter(pos_img, 3),
        # ndimage.gaussian_filter(pos_img, 5),
    ])

    anchor_images = np.stack([[anchor_img] for _ in pos_images])
    neg_images = np.stack([[neg_img] for _ in pos_images])

    loss = batch_update_network(network, loss_fn, optim, anchor_images, pos_images, neg_images)
    return loss

def human_in_the_loop(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at):
    print("HIL TIME!")
    improvements, human_loss, triplet_helpfulness, embedded_archive = hil.humanInput(anchor_dataset, network, optimizer, loss_fn, HIL_archive, random_archive, stop_at)
    print(f"Improvement Count: {improvements}, loss: {human_loss}")

    HIL_archive.save_to_file(f"data/queries/{trial_name}_hil.csv")
    random_archive.save_to_file(f"data/queries/{trial_name}_rand.csv")
    return improvements, human_loss, triplet_helpfulness, embedded_archive

In [6]:
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive
from hil.HIL import HIL
import numpy as np
import time
from torch.utils.tensorboard import SummaryWriter

trial_name = f"{str(int(time.time()))}"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding(out_size=15).to(device)

TRAIN = True
SAVE_CLUSTER_IMAGES = True
SAVE_CLUSTER_MEDOIDS = True
PRETRAINING = True
HUMAN_IN_LOOP = True
SYNTHETIC_HIL = False
EVOLUTION = True
CLUSTER_AND_DISPLAY = True
CONCAT_BVEC_AND_EVEC = True
PASS_THROUGHS = 1
RESULTS_DATA_SIZE = 10000
EVOLUTIONS_PER_EPOCH = 50
K_CLUSTERS = 8
INITIAL_EVOLVES = 3

anchor_dataset = ContinuingDataset("data")
sampled_dataset = SwarmDataset("data/full-dual-sensors", rank=0)
# evolution, _ = ModifiedHaltingEvolution.defaultEvolver(steps=180, n_agents=12, evolve_population=50, seed=None)
evolution, _ = evolution_controller()
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.8)

loss_fn = torch.nn.TripletMarginLoss(margin=15)
hil = HIL(name=trial_name, synthetic=SYNTHETIC_HIL, data_limiter=RESULTS_DATA_SIZE, clusters=K_CLUSTERS)
HIL_archive = DataAggregationArchive()
random_archive = DataAggregationArchive(scalar=True)
EPSILON = 0.5

simulation_time = 0
evolution_time = 0
training_time = 0
hil_time = 0


# TODO: Add Randomly-sampled Contrastive/Triplet Loss
if PRETRAINING:
    network.load_model("two_sensor_subz1")
    print("Pretrained model loaded!")

dataset = anchor_dataset if EVOLUTION else sampled_dataset

if TRAIN:
    writer = SummaryWriter()
    STOP_FLAG = False
    for epoch in range(PASS_THROUGHS + 1):
        if EVOLUTION:
            start_time = time.time()
            evolution.restart_screen()
            for gen in range(EVOLUTIONS_PER_EPOCH if epoch > 0 else INITIAL_EVOLVES):
                # Simulate current population + Save Data
                for i in range(len(evolution.getPopulation())):
                    # The collection of the original behavior vector below is only used to collect data to compare with the baseline
                    visual_behavior, genome, baseline_behavior = evolution.next()
                    dataset.new_entry(visual_behavior, genome, baseline_behavior)
                simulation_time += (time.time() - start_time)

                # Then, evolve
                start_time = time.time()
                embedded_archive = hil.getEmbeddedArchive(dataset, network, concat_behavior=CONCAT_BVEC_AND_EVEC)
                evolution.overwriteArchive(embedded_archive, random_archive)
                embedded_behavior = embedded_archive.archive[-evolution.evolve_config.population:]
                evolution.overwriteBehavior(embedded_behavior)
                evolution.evolve()
                evolution.restart_screen()
                evolution_time += (time.time() - start_time)

                print(f"Evolution complete for e{epoch} and gen{gen}")
                print(f"Size of Evolved Archive: {len(evolution.behavior_discovery.archive.archive)}")

                # Record the accuracy of the medoids with respect to the synthetic policy
                medoid_acc, cluster_acc = 0, 0
                if len(dataset) > 0 and SAVE_CLUSTER_MEDOIDS:
                    if SYNTHETIC_HIL and EVOLUTION:
                        hil.synthetic_knowledge = hil.syntheticBehaviorSpace(dataset)
                        print(f"Synthetic Human Knowledge Size: {len(hil.synthetic_knowledge.labels_)}")
                    medoid_acc, cluster_acc = hil.record_medoids(network, dataset)

                # Cluster current dataset, display clusters, and save for analysis
                if len(dataset) > 0 and SAVE_CLUSTER_IMAGES:
                    hil.embed_and_cluster(network, dataset, auto_quit=True)
                    evolution.restart_screen()

                evolution.saveArchive(trial_name)

        # Human in the Loop determines behavior embedding
        if HUMAN_IN_LOOP and epoch < PASS_THROUGHS:
            start_time = time.time()
            improvements, human_loss, triplet_helpfulness, _ =   human_in_the_loop(
                                                                dataset,
                                                                network,
                                                                optimizer,
                                                                loss_fn,
                                                                HIL_archive,
                                                                random_archive,
                                                                len(dataset)
                                                            )
            hil_time += (time.time() - start_time)

        # Train on past user information
        if TRAIN:
            start_time = time.time()
            loss = None

            average_loss = 50
            loop = 0
            BATCH_SIZE = 5
            network.train()
            while (average_loss > 0.01) and loop < 20:
                loss_sum = 0
                total_loss = 0
                for i, (anchor, pos, neg) in enumerate(HIL_archive):
                    anchor_encoding = dataset[anchor][0]
                    similar_encoding = dataset[pos][0]
                    anti_encoding = dataset[neg][0]
                    loss = train(network, loss_fn, optimizer, anchor_encoding, similar_encoding, anti_encoding)
                    loss_sum += loss.item()
                    total_loss += 1

                    if loss is not None and i > 0 and i % 20 == 0:
                        print(f"Epoch Progress: {(i*100) / len(HIL_archive)}%, Immediate Loss: {loss.item()}")

                average_loss = loss_sum / (total_loss + 1)
                print(f"Loop: {loop}, Average Loss: {average_loss}")
                loop += 1

            print(f"Epoch: {epoch}")
            training_time += (time.time() - start_time)

            writer.add_scalar("Loss/Average", loss_sum / (total_loss + 1), epoch)

        if SAVE_CLUSTER_MEDOIDS:
            writer.add_scalar("Accuracy/MedoidClassification", medoid_acc, epoch)
            writer.add_scalar("Accuracy/RandomSampleClassification", cluster_acc, epoch)

        if HUMAN_IN_LOOP:
            writer.add_scalar("Loss/TripletQuality", triplet_helpfulness, epoch)
            writer.add_scalar("Queries/Total_Human_Queries", epoch*8*9, epoch)
            writer.add_scalar("Queries/Total_Triplets_Generated", len(HIL_archive), epoch)
            writer.add_scalar("Queries/Total_Random_Classes", len(random_archive), epoch)

        if EVOLUTION:
            writer.add_scalar("Novelty/Highest", evolution.behavior_discovery.getBestScore(), epoch)
            writer.add_scalar("Novelty/Average", evolution.behavior_discovery.getAverageScore(), epoch)

        writer.add_scalar("Time/Simulation", simulation_time, epoch)
        writer.add_scalar("Time/HIL", hil_time, epoch)
        writer.add_scalar("Time/Evolution", evolution_time, epoch)
        writer.add_scalar("Time/Training", training_time, epoch)

        scheduler.step()

evolution.close()

HIL Init!
Pretrained model loaded!
<class 'generation.evolution.ModifiedNoveltyArchieve'>
[222.2374462  100.04459865 141.04511362 239.79981469 175.4415071
 177.33573865 270.71765857 156.49053394 230.6029331  457.96620152
 140.26094204 119.61594795 129.44682998 128.06306079 508.85593599
 288.25472235 152.79554795 224.9109829  108.84533078 245.36613291
 219.96892216 118.32192043 946.61468328 107.25828646 246.87798573
 121.10172941 334.98055356 126.6493996  605.54928862 134.31006732
 147.34391689 183.07045384 286.0164349  291.96803754 360.63001128
 130.83413827 491.13739672 125.99038833 151.3901614  226.64990657
 119.915882   115.69840842 554.03895811 159.32603994 215.48677319
 201.37208499 557.25325624 335.75456777 173.93266306 176.02696192
 276.21298147 122.45971645 150.16944498 222.94961936 147.42509247
 124.38069752 156.21399682 174.5577035  159.84383184 178.35268438
 123.70698841 402.49318119 127.29845019 136.73184723 228.54083144
 316.99018161 152.77179053 186.04948301 466.30212319 

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[180.57683716 169.78905891 198.51590835 116.41573319 250.59624616
 316.08326379 125.97141955 302.00933058 126.34040111 181.45362854
 145.47116253 222.3514047   99.05956524 255.46724124 489.26928912
 292.75820089 249.85143125 194.24561991 124.66376853 183.1023254
 157.87185045 167.95477522 284.53341811 239.12737639 237.86777284
 394.58957469 112.04008225  91.18288356 288.48589143 315.50751222
 120.39295608 347.00548429 358.45290527 178.57488402 208.26358203
  94.36696372 214.47529985 130.35167004 361.28263304 102.18305254
 174.05331287 293.39263843 280.60028556 209.48224696 406.36918824
 319.29396223 293.82854731 388.89129944 461.98952914 145.25816769
 194.96418122 525.82874812 203.0425252  166.8064908  242.15206407
 249.53216564 318.21793916 284.66469973 135.31383138 111.52299564
 194.40602311 103.84628417 175.45014135 229.73775721 187.62194363
 250.40097322 118.1437642  171.26064129 186.94053389  94.16847957
 289.90741889 157.2290

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[182.2107561  149.9687623  222.74143291 249.13535976 309.25938813
 214.32520828  84.85689325 112.5541618  190.04032129 121.72887378
 193.47672842 154.71768819 280.1309517  174.90954199 133.42472672
 230.44178571 284.46095739 121.45100552 137.72027945 147.28610174
 296.98078762 177.05851947 261.2629849  201.15831227 171.65316918
 241.30131022 131.42572514  99.91182071 301.76397815 156.00426857
 205.97456141 213.50109642 138.20738381 312.08361164 105.58941767
 110.16366064 220.4799393  211.78460074 242.01755501 287.51793016
  78.18892136 252.6095799  191.11826034 256.33658198 111.60038237
 189.07503065 164.98013676 339.49291256 116.6637724  192.66760972
 168.32009594 110.80622108 158.80101705  95.58588366 166.02012506
 205.21463591 187.1992799  116.74693736 161.47390076 188.52717902
 206.01190082 184.58508523 182.84491148  84.70223967 305.9289664
 248.43252544 178.47815902 110.44375982 200.97310153 209.07111803
  90.19453585 243.7821

...

<class 'generation.evolution.ModifiedNoveltyArchieve'>
[ 9.03801657  7.09377618  8.63198975 19.16049448  8.81032154  9.87994735
  8.0573841   9.35383995  7.23756397  8.63004303  0.96331368  9.32933504
  9.35949925  4.67215776 10.63573003  8.14321689  9.01856071  7.39212407
 15.56427789 10.33449454  8.16427037  8.72677471 10.43204749 12.53377362
  6.72023117 11.177366    7.79641318  7.52138467  7.79034867 10.6232345
  9.12160453  0.84210704  8.24846236  6.87546468 13.20562166  7.64973203
 10.39868442  9.02914372  7.56268161 10.46420373 11.83031899  8.39317567
 11.5655097  10.52208774  7.62712761  7.22903484 12.3065292  18.66134818
  5.5511155   7.70394533 17.67511864 13.02303739 10.43960923 10.244424
  4.63634098 11.10150122  8.19622852  5.40169637  8.74684654 10.10923027
  3.78583525 10.87404307 12.15352191  8.63883963  9.52629089 10.34568402
  9.49635189 10.59432642  7.54111623 10.64772115  3.61799451  7.41333649
 12.28383107  9.39058221 14.44886851 26.94769296  9.37058659 10.04138768

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


<class 'generation.evolution.ModifiedNoveltyArchieve'>
[17.21191768  7.33310735  4.02669772  8.83068667 16.23469     1.16724301
  7.72468661  9.17740598  9.74314243  8.83158782  7.64587119  7.85334225
  6.39082242  9.91321064  8.02313049 13.05436985  6.45390074  8.27948497
  9.98907512  0.92052789  9.70331921 11.57436576  7.32692142  6.13681033
  2.45019082  7.21680159 11.58598753  1.3660694   6.91386767  7.41615695
  8.15378076  8.05099388 10.67816221  8.41652682  0.81808644 10.30220901
  1.9130103  11.52641098 11.31765866 13.84949449 15.48803787  7.61149477
 17.18227957  9.03803001  7.75633504  5.63227146  4.22212499  1.42625785
 14.14983295  8.75408142 11.12143294 14.00262766 39.78938542  8.47311994
 29.27992713  8.09350672  6.25486154  7.127885    7.97676663  9.11028469
 12.77864692  9.83166716  6.47828579  9.71222559  8.3739302   7.35522169
  9.40868998 15.47720458 10.78875724  9.70198954  9.64164641  7.70947047
  5.39655317  7.41916063 13.26881176 12.61739358  8.30867557  6.25465

/home/connor/Desktop/research/SwarmNoveltyNetwork/.env/lib/python3.10/site-packages/sklearn/manifold/_t_sne.py:996: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(


Epoch Progress: 11.11111111111111%, Immediate Loss: 0.0
Epoch Progress: 22.22222222222222%, Immediate Loss: 0.0
Epoch Progress: 33.333333333333336%, Immediate Loss: 0.0
Epoch Progress: 44.44444444444444%, Immediate Loss: 0.0
Epoch Progress: 55.55555555555556%, Immediate Loss: 0.0
Epoch Progress: 66.66666666666667%, Immediate Loss: 0.0
Epoch Progress: 77.77777777777777%, Immediate Loss: 0.0
Epoch Progress: 88.88888888888889%, Immediate Loss: 0.0
Loop: 0, Average Loss: 0.0
Epoch: 1


In [ ]:
# Evolve and Display?
print(len(evolution.archive.archive))

In [ ]:
import pygame
pygame.quit()

# Pretraining

In [16]:
# Save Model
import torch
from data.swarmset import ContinuingDataset, SwarmDataset
from networks.embedding import NoveltyEmbedding
from networks.archive import DataAggregationArchive

PRETRAINING = True
target = 0.01
loss = 50
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding(out_size=15).to(device)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.9)
sampled_dataset = SwarmDataset("data/full-dual-sensors", rank=0)

t_1 = time.time()
if PRETRAINING:
    epochs = 0
    while loss > target:
        loss = pretraining(sampled_dataset, network, loss_fn, optimizer, data_cutoff=None, data_size=1000)
        print(f"Epoch {epochs}, loss: {loss}")
        epochs += 1
        scheduler.step()

print(f"Total Pre-training Time: {time.time() - t_1}")

network.save_model()

/tmp/ipykernel_4115242/3999654231.py:27: DeprecationWarning: This function is deprecated. Please call randint(0, 9999 + 1) instead
  samples = np.random.random_integers(0, data_cutoff, (data_size, 2))


Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised Training.. 65.0
Unsupervised Training.. 70.0
Unsupervised Training.. 75.0
Unsupervised Training.. 80.0
Unsupervised Training.. 85.0
Unsupervised Training.. 90.0
Unsupervised Training.. 95.0
Epoch 0, loss: 4.559638996657006
Unsupervised Training.. 0.0
Unsupervised Training.. 5.0
Unsupervised Training.. 10.0
Unsupervised Training.. 15.0
Unsupervised Training.. 20.0
Unsupervised Training.. 25.0
Unsupervised Training.. 30.0
Unsupervised Training.. 35.0
Unsupervised Training.. 40.0
Unsupervised Training.. 45.0
Unsupervised Training.. 50.0
Unsupervised Training.. 55.0
Unsupervised Training.. 60.0
Unsupervised T

In [ ]:
network.save_model()

In [ ]:
print(optimizer.lr)

In [ ]:

import torch
from networks.embedding import NoveltyEmbedding
from NovelSwarmBehavior.novel_swarms.novelty.NoveltyArchive import NoveltyArchive
from NovelSwarmBehavior.novel_swarms.config.ResultsConfig import ResultsConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from data.swarmset import DataBuilder

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
network = NoveltyEmbedding().to(device)
network.load_model("trialA-10-18-2022")
# anchor_dataset = SwarmDataset("data/full", rank=0)

WRITE_OUT = True
if WRITE_OUT:
    network.eval()
    test_archive = NoveltyArchive()
    for i in range(len(anchor_dataset)):
        anchor_encoding, genome, _ = anchor_dataset[i]
        anchor_encoding = torch.from_numpy(anchor_encoding).to(device).float()
        embedding = network(anchor_encoding.unsqueeze(0)).squeeze(0).cpu().detach().numpy()
        test_archive.addToArchive(embedding, genome)

In [ ]:
from ui.clustering_gui import ClusteringGUI
import pygame

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=test_archive, k_clusters=6, world_config=world_config, tsne_perplexity=16, tsne_early_exaggeration=12, skip_tsne=False)
gui = ClusteringGUI(config)
gui.displayGUI()
# results(config)
pygame.quit()

# Clustering + Analysis

In [ ]:
from ui.clustering_gui import ClusteringGUI

# Cluster over saved behaviors
archive = NoveltyArchive()
for i, (_, genome, behavior, _, _, _, _) in enumerate(anchor_dataset):
    archive.addToArchive(vec=behavior, genome=genome)

agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT
world_config = ConfigurationDefaults.RECTANGULAR_WORLD
world_config.addAgentConfig(agent_config)
config = ResultsConfig(archive=archive, k_clusters=7, world_config=world_config, tsne_perplexity=20, tsne_early_exaggeration=2, skip_tsne=False)

gui = ClusteringGUI(config)
gui.displayGUI()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(human_l_hist, "b", label='Human Loss')
plt.ylabel("Loss")
plt.xlabel("Time (Epochs)")
plt.legend()